## Model interpretability

In this notebook you will learn how to set up inait's explanation tool.

We will use a electric power dataset as a companion dataset but feel free to use your own data!

### Prerequisites

Let's import the relevant packages and add load the credentials that you've inserted in `credentials.txt`.

In [ ]:
import sys
import pathlib

import pandas as pd

sys.path.append(pathlib.Path().resolve().parent.as_posix())

from inait import predict, plot, load_credentials

base_url, auth_key = load_credentials("../credentials.txt")

### First run the prediction model

We will use ...

To forecast using inait models you need to specifiy the dataset using `data_path`, how many steps in the future you want to forecast (the so-called `forecasting_horizon`), and how many steps in the past you want to use to make a prediction (`observation_length`). 

In [ ]:
### Load the data
data_path = "../data/demo_power.csv"
data = pd.read_csv(
    data_path, index_col=0
)  # dataset must have a valid datetime index with fixed frequency

# Configure prediction parameters
target_columns = [
    "DE_Spot_EPEX_1H_A"
]  # List of target columns to predict in the dataset
feature_columns = [
    "DE_Residual_Load_15M_A_AVG",
    "DE_Consumption_15M_A_AVG",
]  # Optional: List of feature columns to use for prediction
forecasting_horizon = 12  # Predict 12 time steps ahead
observation_length = 12  # Use last 12 time steps as historical context

In [ ]:
prediction, session_id = predict(
    base_url=base_url,
    auth_key=auth_key,
    data_path=data_path,
    target_columns=target_columns,
    feature_columns=feature_columns,
    forecasting_horizon=forecasting_horizon,
    observation_length=observation_length,
)

prediction

In [ ]:
plot(
    historical_data=data.loc[:, target_columns],
    predicted_data=prediction,
    observation_length=24,
)

You can now try it out with different `forecasting_horizon` or `observation_length` parameters, or use you own dataset!

In [ ]:
from inait import create_explanation_payload, make_request

payload = create_explanation_payload(
    session_id=session_id, cutoff_days="1960-12-31 00:00:00"
)
explanation_response = make_request(
    base_url + "/explanation", payload, auth_key=auth_key
)
df = explanation_response["response"]["data"]["1960-12-31 00:00:00"][
    "1961-01-31 00:00:00"
]

df = pd.DataFrame([df])
df = df.iloc[:, :-1]
df = df.T

df